In [1]:
# ATSIT 2019: dimensional speech emotion recognition from text feature

# uncomment these to run on CPU only
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]=""

import numpy as np
import matplotlib.pyplot as plt
import pickle
import keras.backend as K
import pandas as pd
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, LSTM, TimeDistributed, Bidirectional, Embedding,GRU, Dropout, Flatten, concatenate, Convolution1D, BatchNormalization
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from keras.layers.convolutional import Conv2D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
import codecs
from keras.optimizers import SGD, Adam, RMSprop
import io
import random as rn
import tensorflow as tf
from keras.layers.core import Dense, Activation
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)
# load feature and labels
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')
# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

# standardization
scaled_feature = False
if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1, keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss
import pickle
with open('../../dimensionalSM/IEMOCAP_full_release/data_collected_333.pickle', 'rb') as handle:
    data2 = pickle.load(handle)
    
x_train_mocap = []
counter = 0
for ses_mod in data2:
    x_head = ses_mod['mocap_head']
    if(x_head.shape != (200,18)):
        x_head = np.zeros((200,18))   
    x_head[np.isnan(x_head)]=0
    x_hand = ses_mod['mocap_hand']
    if(x_hand.shape != (200,6)):
        x_hand = np.zeros((200,6))   
    x_hand[np.isnan(x_hand)]=0
    x_rot = ses_mod['mocap_rot']
    if(x_rot.shape != (200,165)):
        x_rot = np.zeros((200,165))   
    x_rot[np.isnan(x_rot)]=0
    x_mocap = np.concatenate((x_head, x_hand), axis=1)
    x_mocap = np.concatenate((x_mocap, x_rot), axis=1)
    x_train_mocap.append( x_mocap )
    
x_train_mocap = np.array(x_train_mocap)
#x_train_mocap = x_train_mocap.reshape(-1,200,189,1)
x_train_mocap.shape
# remove outlier
# outlier = np.array([1674, 3427, 5086, 5093, 5096, 5104, 7821])
# mask = np.ones(len(vad), np.bool)
# mask[outlier] = 0
# vad = vad[mask]
# x_train_mocap = x_train_mocap[mask]


Using TensorFlow backend.


(10039, 200, 189)

In [2]:
# API model, if use RNN, first two rnn layer must return_sequences=True
def api_model(a,b,c):
    
    input_mocap = Input(shape=(200,189))
    model_mocap0 = BatchNormalization()(input_mocap)
    model_mocap1 = Convolution1D(64, 3, border_mode='same')(model_mocap0)
    model_mocap2 = Dropout(0.2)(model_mocap1)
    model_mocap3 = Activation('relu')(model_mocap2)
    model_mocap4 = Convolution1D(128, 3, border_mode='same')(model_mocap3)
    model_mocap5 = Dropout(0.2)(model_mocap4)
    model_mocap6 = Activation('relu')(model_mocap5)
    model_mocap7 = Convolution1D(256, 3, border_mode='same')(model_mocap6)
    model_mocap8 = Dropout(0.2)(model_mocap7) 
    model_mocap9 = Activation('relu')(model_mocap8)
    model_mocap10 = Convolution1D(128, 3, border_mode='same')(model_mocap9)
    model_mocap11 = Dropout(0.2)(model_mocap10)  
    model_mocap12 = Activation('relu')(model_mocap11)
    model_mocap13 = Convolution1D(64, 3, border_mode='same')(model_mocap12)
    model_mocap14 = Dropout(0.2)(model_mocap13) 
    model_mocap15 = Activation('relu')(model_mocap14)
    model_mocap = Flatten()(model_mocap15) 
    #model_mocap = Dropout(dropout_rate)(model_mocap16)

    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_mocap) for name in target_names]

    model = Model(input_mocap, model_combined) 

    model.compile(loss=ccc_loss, 
                  loss_weights={'v': a, 'a': b, 'd': c},
                  optimizer='rmsprop', metrics=[ccc])
    return model

#def main(alpha, beta, gamma):
#model = api_model(1, 1, 1)
#odel = api_model(0.1, 0.5, 0.4)
#model.summary()
# main function for for LOSO


In [3]:
def main():
    model = api_model(1,1,1)
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model.fit( x_train_mocap[:8000], 
                      vad[:8000].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model.evaluate( x_train_mocap[8000:], vad[8000:].T.tolist())
    print(metrik[-3:])
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.1, 0.5, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main()
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('mocap_CNN1D_8000-batch.csv', header=['time', 'v','a','d', 'ave'])


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


/root/anaconda3/envs/ser36/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same")`
  
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, 3, padding="same")`
  if __name__ == '__main__':
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(256, 3, padding="same")`
  if sys.path[0] == '':
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, 3, padding="same")`
  from ipykernel import kernelapp as app
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same")`



Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 15s 2ms/step - loss: 2.1236 - v_loss: 0.5644 - a_loss: 0.7798 - d_loss: 0.7795 - v_ccc: 0.4356 - a_ccc: 0.2202 - d_ccc: 0.2205 - val_loss: 2.5796 - val_v_loss: 0.8621 - val_a_loss: 0.8536 - val_d_loss: 0.8639 - val_v_ccc: 0.1379 - val_a_ccc: 0.1464 - val_d_ccc: 0.1361
Epoch 2/50
6400/6400 [==============================] - 14s 2ms/step - loss: 1.7002 - v_loss: 0.4188 - a_loss: 0.6399 - d_loss: 0.6414 - v_ccc: 0.5812 - a_ccc: 0.3601 - d_ccc: 0.3586 - val_loss: 2.5111 - val_v_loss: 0.8872 - val_a_loss: 0.7934 - val_d_loss: 0.8304 - val_v_ccc: 0.1128 - val_a_ccc: 0.2066 - val_d_ccc: 0.1696
Epoch 3/50
6400/6400 [==============================] - 15s 2ms/step - loss: 1.5435 - v_loss: 0.3859 - a_loss: 0.5732 - d_loss: 0.5844 - v_ccc: 0.6141 - a_ccc: 0.4268 - d_ccc: 0.4156 - val_loss: 2.4582 - val_v_loss: 0.8274 - val_a_loss: 0.7954 - val_d_loss: 0.8354 - val_v_ccc: 0.1726 - val_a_ccc: 0.2

6400/6400 [==============================] - 14s 2ms/step - loss: 1.1776 - v_loss: 0.3041 - a_loss: 0.4451 - d_loss: 0.4284 - v_ccc: 0.6959 - a_ccc: 0.5549 - d_ccc: 0.5716 - val_loss: 2.3679 - val_v_loss: 0.7906 - val_a_loss: 0.7528 - val_d_loss: 0.8245 - val_v_ccc: 0.2094 - val_a_ccc: 0.2472 - val_d_ccc: 0.1755
Epoch 26/50
2039/2039 [==============================] - 1s 568us/step
[0.30995774269104004, 0.21957427263259888, 0.23318275809288025]
0


/root/anaconda3/envs/ser36/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same")`
  
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, 3, padding="same")`
  if __name__ == '__main__':
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(256, 3, padding="same")`
  if sys.path[0] == '':
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, 3, padding="same")`
  from ipykernel import kernelapp as app
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same")`


...

Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 19s 3ms/step - loss: 2.1056 - v_loss: 0.5511 - a_loss: 0.7859 - d_loss: 0.7686 - v_ccc: 0.4489 - a_ccc: 0.2141 - d_ccc: 0.2314 - val_loss: 2.6826 - val_v_loss: 0.8641 - val_a_loss: 0.8724 - val_d_loss: 0.9461 - val_v_ccc: 0.1359 - val_a_ccc: 0.1276 - val_d_ccc: 0.0539
Epoch 2/50
6400/6400 [==============================] - 16s 2ms/step - loss: 1.7352 - v_loss: 0.4182 - a_loss: 0.6573 - d_loss: 0.6598 - v_ccc: 0.5818 - a_ccc: 0.3427 - d_ccc: 0.3402 - val_loss: 2.5057 - val_v_loss: 0.9145 - val_a_loss: 0.7793 - val_d_loss: 0.8119 - val_v_ccc: 0.0855 - val_a_ccc: 0.2207 - val_d_ccc: 0.1881
Epoch 3/50
6400/6400 [==============================] - 16s 3ms/step - loss: 1.5519 - v_loss: 0.3930 - a_loss: 0.5801 - d_loss: 0.5788 - v_ccc: 0.6070 - a_ccc: 0.4199 - d_ccc: 0.4212 - val_loss: 2.5672 - val_v_loss: 0.9153 - val_a_loss: 0.8264 - val_d_loss: 0.8256 - val_v_ccc: 0.0847 - val_a_ccc: 0.17

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6400/6400 [==============================] - 16s 3ms/step - loss: 1.4277 - v_loss: 0.3532 - a_loss: 0.5449 - d_loss: 0.5295 - v_ccc: 0.6468 - a_ccc: 0.4551 - d_ccc: 0.4705 - val_loss: 2.3869 - val_v_loss: 0.8601 - val_a_loss: 0.7377 - val_d_loss: 0.7891 - val_v_ccc: 0.1399 - val_a_ccc: 0.2623 - val_d_ccc: 0.2109
Epoch 7/50
6400/6400 [==============================] - 16s 3ms/step - loss: 1.4038 - v_loss: 0.3537 - a_loss: 0.5308 - d_loss: 0.5193 - v_ccc: 0.6463 - a_ccc: 0.4692 - d_ccc: 0.4807 - val_loss: 2.4759 - val_v_loss: 0.8735 - val_a_loss: 0.7883 - val_d_loss: 0.8141 - val_v_ccc: 0.1265 - val_a_ccc: 0.2117 - val_d_ccc: 0.1859
Epoch 8/50
6400/6400 [==============================] - 16s 3ms/step - loss: 1.3829 - v_loss: 0.3497 - a_loss: 0.5191 - d_loss: 0.5141 - v_ccc: 0.6503 - a_ccc: 0.4809 - d_ccc: 0.4859 - val_loss: 2.4273 - val_v_loss: 0.8578 - val_a_loss: 0.7816 - val_d_loss: 0.7879 - val_v_ccc: 0.1422 - val_a_ccc: 0.2184 - val_d_ccc: 0.2121
Epoch 9/50
6400/6400 [==============

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2039/2039 [==============================] - 2s 749us/step
[0.31786975264549255, 0.28852322697639465, 0.23481248319149017]
19
